## Mathematical Animations WITH EASE | Ep. 04: Updaters

This notebook contains the examples discussed in [this video](https://youtu.be/vUIfNN6Bs_4). The cells below contain the final state of the corresponding scene from the video. Feel free to modify the code below and play around, your changes are only local and just visible to you. To add some more new cells, you can use the "+" icon above.

To run cells, click on them so that they are in focus, then either the "Run" button above, or hit `<Shift + Enter>`.

In [ ]:
from manim import *

These are some useful basic settings for Jupyter notebooks. You can use them by executing the cell, or just skip them.

In [ ]:
config.media_width = "80%"
config.verbosity = "WARNING"

#### Examples using Updater Functions

In [ ]:
%%manim -qm MovingLabel

class MovingLabel(Scene):
    def construct(self):
        blue_dot = Dot(color=BLUE)
        dot_label = Text("Hello dot!").next_to(blue_dot, UP)
        dot_label.add_updater(
            lambda mobject: mobject.next_to(blue_dot, UP)
        )
        self.add(blue_dot, dot_label)
        self.play(blue_dot.animate.shift(RIGHT))
        self.play(blue_dot.animate.scale(10))
        self.play(blue_dot.animate.move_to([-2, -2, 0]))

In [ ]:
%%manim -qm --disable_caching AllUpdaterTypes

class AllUpdaterTypes(Scene):
    def construct(self):
        red_dot = Dot(color=RED).shift(LEFT)
        pointer = Arrow(ORIGIN, RIGHT).next_to(red_dot, LEFT)
        pointer.add_updater( # place arrow left of dot
            lambda mob: mob.next_to(red_dot, LEFT)
        )
        
        def shifter(mob, dt): # make dot move 2 units RIGHT/sec
            mob.shift(2*dt*RIGHT)
        red_dot.add_updater(shifter)
        
        def scene_scaler(dt): # scale mobjects depending on distance to origin
            for mob in self.mobjects:
                mob.set(width=2/(1 + np.linalg.norm(mob.get_center())))
        self.add_updater(scene_scaler)
        
        self.add(red_dot, pointer)
        # scene has to update initially to fix first frame:
        # first mobject updaters are called, then scene updaters
        self.update_self(0)
        self.wait(5)
        

#### Combining Updater Functions and Animations

In [ ]:
%%manim -qm --disable_caching UpdaterAndAnimation

class UpdaterAndAnimation(Scene):
    def construct(self):
        red_dot = Dot(color=RED).shift(LEFT)
        rotating_square = Square()
        rotating_square.add_updater(lambda mob, dt: mob.rotate(dt*PI))
        
        def shifter(mob, dt): # make dot move 2 units RIGHT/sec
            mob.shift(2*dt*RIGHT)
        red_dot.add_updater(shifter)
        
        self.add(red_dot, rotating_square)
        self.wait(1)
        red_dot.suspend_updating()
        self.wait(1)
        self.play(
            red_dot.animate.shift(UP),
            rotating_square.animate.move_to([-2, -2, 0])
        )
        self.wait(1)

#### `ValueTracker`

In [ ]:
%%manim -qm ValueTrackerMove

class ValueTrackerMove(Scene):
    def construct(self):
        line = NumberLine(x_range=[-5, 5])
        position = ValueTracker(0)
        pointer = Vector(DOWN)
        pointer.add_updater(
            lambda mob: mob.next_to(
                line.number_to_point(position.get_value()), UP
            )
        )
        pointer.update()
        self.add(line, pointer)
        self.wait()
        self.play(position.animate.set_value(4))
        self.play(position.animate.set_value(-2))

In [ ]:
%%manim -qm ValueTrackerPlot

class ValueTrackerPlot(Scene):
    def construct(self):
        a = ValueTracker(1)
        ax = Axes(x_range=[-2, 2, 1], y_range=[-8.5, 8.5, 1], x_length=4, y_length=6)
        parabola = ax.plot(lambda x: a.get_value() * x**2, color=RED)
        parabola.add_updater(
            lambda mob: mob.become(ax.plot(lambda x: a.get_value() * x**2, color=RED))
        )
        a_number = DecimalNumber(
            a.get_value(),
            color=RED,
            num_decimal_places=3,
            show_ellipsis=True
        )
        a_number.add_updater(
            lambda mob: mob.set_value(a.get_value()).next_to(parabola, RIGHT)
        )
        self.add(ax, parabola, a_number)
        self.play(a.animate.set_value(2))
        self.play(a.animate.set_value(-2))
        self.play(a.animate.set_value(1))